# 03– Optimización multi-período

Objetivo

Optimizar la política de pedidos considerando varios períodos (semanas o meses) y restricciones de inventario, usando Programación Lineal (PL) con PuLP.

In [1]:
import importlib
import subprocess
import sys

def install_and_import(package, import_name=None):
    """Instala un paquete si no está presente."""
    import_name = import_name or package
    try:
        importlib.import_module(import_name)
        print(f"{package} ya está instalado.")
    except ImportError:
        print(f" Instalando {package}...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])
        print(f"{package} instalado correctamente.")

# ----------------------------------------------------------
# Librerías requeridas por los notebooks prescriptivos
# ----------------------------------------------------------
dependencies = {
    "pandas": "pandas",
    "numpy": "numpy",
    "pulp": "pulp",
    "statsmodels": "statsmodels",
    "matplotlib": "matplotlib",
    "seaborn": "seaborn",
    "plotly": "plotly"
}

for pkg, import_name in dependencies.items():
    install_and_import(pkg, import_name)

print("\n Todas las dependencias están listas.\n")


pandas ya está instalado.
numpy ya está instalado.
pulp ya está instalado.
statsmodels ya está instalado.
matplotlib ya está instalado.
statsmodels ya está instalado.
matplotlib ya está instalado.
seaborn ya está instalado.
plotly ya está instalado.

 Todas las dependencias están listas.

seaborn ya está instalado.
plotly ya está instalado.

 Todas las dependencias están listas.



In [2]:
# 1 Importar librerías
import pandas as pd
from pulp import LpMinimize, LpProblem, LpVariable, lpSum, value


In [3]:
# 2 Cargar datos base
ventas = pd.read_csv("resultados_modelo_sQ.csv")
print(" Datos cargados:", ventas.shape)

 Datos cargados: (5000, 24)


In [4]:
# 3 Definir parámetros de optimización
# Cargamos la demanda por periodo (semanas) desde el forecast para construir
# un vector de demanda por producto y periodo en lugar de una media escalar.
forecast = pd.read_csv("resultados_forecast.csv")
# normalizar columna semana si tiene otro nombre
if 'semana_a_futuro' in forecast.columns and 'semana' not in forecast.columns:
    forecast = forecast.rename(columns={'semana_a_futuro':'semana'})

# calcular lista de periodos y lista de productos
periodos = sorted(forecast['semana'].dropna().unique().astype(int).tolist())
# USAR TODOS LOS PRODUCTOS
productos = sorted(ventas['sku'].unique())
print(f"Usando {len(productos)} productos y {len(periodos)} periodos")

costo_pedido = 50
costo_mant = 0.25

# construir diccionario de demanda por producto y periodo: demanda[p][t]
demanda_df = forecast.groupby(['sku','semana'])['prediccion_ARIMA'].sum().reset_index()
demanda = {p: {t: 0.0 for t in periodos} for p in productos}
for _, row in demanda_df.iterrows():
    sku = row['sku']
    sem = int(row['semana'])
    if sku in demanda:
        demanda[sku][sem] = float(row['prediccion_ARIMA'])


Usando 100 productos y 8 periodos


In [5]:
# 4 Crear modelo
modelo = LpProblem("Optimización_Multi_Periodo", LpMinimize)

# Variables: pedidos y stock (usar lista de periodos)
pedidos = LpVariable.dicts("pedido", (productos, periodos), lowBound=0)
stock = LpVariable.dicts("stock", (productos, periodos), lowBound=0)


In [6]:
# 5 Función objetivo: minimizar costo total REALISTA
# Penalizar fuertemente el exceso de stock para evitar pedidos masivos

# Función objetivo mejorada:
# - Costo fijo reducido por pedido (incentiva pedidos pequeños frecuentes)
# - Penalización mayor por mantener stock (desincentiva acumulación)
modelo += lpSum([
    2.0 * pedidos[p][t] + costo_mant * 2 * stock[p][t]  # Duplicar penalización de stock
    for p in productos for t in periodos
])

print("Modelo creado con función objetivo realista (penaliza exceso de stock)")


Modelo creado con función objetivo realista (penaliza exceso de stock)


In [ ]:
# =====================================================
# STOCK DE SEGURIDAD ACTIVADO 
# =====================================================
# Stock de seguridad variable por producto usando datos históricos
# Nivel de servicio: 95% (Z = 1.65)

if 'desviacion_demanda' in ventas.columns:
    print("Aplicando stock de seguridad estadístico (95% nivel de servicio)...")
    
    contador = 0
    for p in productos:
        # Obtener desviación estándar del producto
        producto_data = ventas[ventas['sku'] == p]
        if not producto_data.empty:
            desv = producto_data['desviacion_demanda'].mean()
            # Stock de seguridad = Z × σ (Z=1.65 para 95% nivel de servicio)
            stock_seguridad = 1.65 * desv if not pd.isna(desv) else 10.0
            
            # Aplicar restricción para todos los periodos
            for t in periodos:
                modelo += stock[p][t] >= stock_seguridad
            
            contador += 1
    
    print(f" Stock de seguridad configurado para {contador} productos")
    print(f" Esto garantiza 95% de nivel de servicio ante variabilidad de demanda")
else:
    print(" Columna 'desviacion_demanda' no encontrada. Aplicando stock mínimo del 10%...")
    # Fallback: usar stock mínimo fijo
    for p in productos:
        demanda_promedio = sum(demanda[p].values()) / len(periodos) if len(periodos) > 0 else 0
        stock_minimo = max(demanda_promedio * 0.1, 5.0)  # Mínimo 5 unidades
        for t in periodos:
            modelo += stock[p][t] >= stock_minimo
    
    print(" Stock mínimo del 10% aplicado como alternativa")

Aplicando stock de seguridad estadístico (95% nivel de servicio)...
 Stock de seguridad configurado para 100 productos
 Esto garantiza 95% de nivel de servicio ante variabilidad de demanda
 Stock de seguridad configurado para 100 productos
 Esto garantiza 95% de nivel de servicio ante variabilidad de demanda


In [ ]:
# =====================================================
# 6️⃣ RECREAR MODELO Y AGREGAR RESTRICCIONES
# =====================================================
from pulp import value, PULP_CBC_CMD, LpStatus, LpVariable, LpProblem, LpMinimize, lpSum
import time, sys, threading
from datetime import datetime

# IMPORTANTE: Re-crear el modelo desde cero para asegurar que las restricciones
# se apliquen correctamente (las modificaciones incrementales no funcionan)

print(" Re-creando modelo con restricciones actualizadas...")

# 1. Crear modelo nuevo
modelo = LpProblem("Optimización_Multi_Periodo_v2", LpMinimize)

# 2. Crear variables
pedidos = LpVariable.dicts("pedido", (productos, periodos), lowBound=0)
stock = LpVariable.dicts("stock", (productos, periodos), lowBound=0)

# 3. Función objetivo (penalizar exceso de stock)
modelo += lpSum([
    2.0 * pedidos[p][t] + 0.5 * 2 * stock[p][t]
    for p in productos for t in periodos
])

# 4. Stock de seguridad
if 'desviacion_demanda' in ventas.columns:
    for p in productos:
        producto_data = ventas[ventas['sku'] == p]
        if not producto_data.empty:
            desv = producto_data['desviacion_demanda'].iloc[0]
            stock_seguridad = 1.65 * desv  # 95% nivel de servicio
            for t in periodos:
                modelo += stock[p][t] >= stock_seguridad

# 5. Restricciones de balance por periodo (usando demanda[p][t])
for p in productos:
    # Calcular demanda del periodo actual para cada producto
    for idx, t in enumerate(periodos):
        demanda_periodo_actual = demanda[p].get(t, 0)
        
        if idx == 0:
            # Periodo inicial: stock = pedido - demanda
            modelo += stock[p][t] == pedidos[p][t] - demanda_periodo_actual
        else:
            # Periodos siguientes: stock = stock_anterior + pedido - demanda
            modelo += stock[p][t] == stock[p][periodos[idx-1]] + pedidos[p][t] - demanda_periodo_actual
        
        # Evitar stock negativo
        modelo += stock[p][t] >= 0
        
        # RESTRICCIÓN CLAVE: Limitar pedido a 2x la demanda del periodo ACTUAL
        # Esto permite cubrir el periodo actual + algo de buffer, pero no acumular para todo el horizonte
        pedido_maximo = max(demanda_periodo_actual * 2.0, 5.0)
        modelo += pedidos[p][t] <= pedido_maximo
        
        # RESTRICCIÓN ADICIONAL: Limitar stock acumulado a 3x la demanda del periodo
        # Evita acumulación excesiva
        stock_maximo = max(demanda_periodo_actual * 3.0, 10.0)
        modelo += stock[p][t] <= stock_maximo

# =====================================================
# 7 Resolver con progreso (REDUCIDO A 3 MINUTOS)
# =====================================================
print(f"\nInicio del proceso: {datetime.now()}")
print(f"Optimizando {len(productos)} productos × {len(periodos)} periodos")

# REDUCIR TIMEOUT A 180 SEGUNDOS (3 MIN) PARA PRUEBA
solver = PULP_CBC_CMD(msg=True, timeLimit=180, threads=6)

# --- Barra de progreso en paralelo ---
def barra_progreso(total_tiempo=180, intervalo=5):
    pasos = total_tiempo // intervalo
    for i in range(pasos):
        time.sleep(intervalo)
        progreso = (i + 1) / pasos * 100
        sys.stdout.write(f"\rProgreso estimado: {progreso:5.1f}%")
        sys.stdout.flush()
    print()

progreso_thread = threading.Thread(target=barra_progreso, args=(180, 5))
progreso_thread.start()

inicio = datetime.now()
status = modelo.solve(solver)
fin = datetime.now()

progreso_thread.join()

print(f"\nEstado final del modelo: {LpStatus[status]}")
print(f"Duración total: {fin - inicio}")

# =====================================================
# 8 Exportar resultados
# =====================================================
resultados = [
    {"sku": p, "periodo": t, "pedido": value(pedidos[p][t]), "stock": value(stock[p][t])}
    for p in productos for t in periodos
]

df_result = pd.DataFrame(resultados)

# Renombrar columnas para claridad
df_result = df_result.rename(columns={'periodo': 'semana'})

df_result.to_csv("resultados_multi_periodo.csv", index=False)
print("\n Archivo 'resultados_multi_periodo.csv' generado correctamente.")

print("\n Distribución de pedidos por periodo:")
print(df_result.groupby('semana')['pedido'].sum())

print("\n Primeras filas:")
print(df_result.head(20))

# Estadísticas adicionales
print("\n Estadísticas generales:")
print(f"   Pedido total: {df_result['pedido'].sum():,.0f} unidades")
print(f"   Stock promedio: {df_result['stock'].mean():,.2f} unidades")
print(f"   Pedido promedio por periodo: {df_result.groupby('semana')['pedido'].sum().mean():,.0f}")

🔄 Re-creando modelo con restricciones actualizadas...

Inicio del proceso: 2025-11-24 00:28:12.358844
Optimizando 100 productos × 8 periodos

Inicio del proceso: 2025-11-24 00:28:12.358844
Optimizando 100 productos × 8 periodos
Progreso estimado: 100.0%

Estado final del modelo: Optimal
Duración total: 0:00:00.137632

✅ Archivo 'resultados_multi_periodo.csv' generado correctamente.

📊 Distribución de pedidos por periodo:
semana
1    1784.795012
2    1617.630000
3    1619.990000
4    1619.230000
5    1619.690000
6    1619.580000
7    1619.680000
8    1619.650000
Name: pedido, dtype: float64

📋 Primeras filas:
         sku  semana     pedido     stock
0   HM000001       1  17.662860  1.922861
1   HM000001       2  16.070000  1.922861
2   HM000001       3  15.940000  1.922861
3   HM000001       4  15.970000  1.922861
4   HM000001       5  15.970000  1.922861
5   HM000001       6  15.960000  1.922861
6   HM000001       7  15.960000  1.922861
7   HM000001       8  15.960000  1.922861
8   HM